In [1]:
#Any code, applications, scripts, templates, proofs of concept, documentation and other items provided by AWS under this SOW are “AWS Content,” as defined in the Agreement, and are provided for illustration purposes only. All such AWS Content is provided solely at the option of AWS, and is subject to the terms of the Addendum and the Agreement. Customer is solely responsible for using, deploying, testing, and supporting any code and applications provided by AWS under this SOW.

In [157]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
import random
import sys
import os
import json
from pprint import pprint
import pandas as pd

#images path
os.chdir('/home/ec2-user/table_detection/json2df/VOC2007/JPEGImages')
# path to dataframe
df_path = '/home/ec2-user/table_detection/json2df/meta_dataframe.csv'
new_df_path = '/home/ec2-user/table_detection/json2df/meta_dataframe_updated.csv'

# loop over all the files and slice them 
def slice_image_loop(outdir):
    df = pd.read_pickle(df_path)
    for path, subdirs, files in os.walk(outdir):
        if not os.path.exists(str(path)+'_sliced'):
            os.makedirs(str(path)+'_sliced')
        for name in files:
            if name.endswith('.jpeg'): # change outdir
                for cut_side in ["left","right","bottom","top"]: # loop over sidi
                    for i in np.arange(0, 1.95, 0.15): #for i in np.arange(0, 5, 0.015) # loop over cutting %
                        cut_percent = i 
                        image_path = os.path.join(str(path)+'/'+name)
                        image = cv2.imread(image_path,0)
                        corped_img = slice_image(image, cut_percent, part=cut_side)
                        new_file_name = str(path)+'_sliced/'+cut_side+'_'+str(i)+'_'+name
                        # add a line to the data frame:
                        temp_df_row = df.loc[df[0] == name]
                        flag = 0
                        column_number = 1
                        temp_list = temp_df_row[column_number]
                        temp_df_row.set_value(temp_df_row.index[0], 0, cut_side+'_'+str(i)+'_'+name)
                        #print("temp_df_row[column_number]: ",temp_df_row[column_number])
                        temp_list = temp_list.tolist()
                        temp_list = temp_list[0]            
                        while flag==0:
                            if (cut_side=='left'):
                                temp_list[0] = temp_list[0] - image.shape[1]*cut_percent/100
                            elif (cut_side=='top'):
                                temp_list[1] = temp_list[1] - image.shape[0]*cut_percent/100
                            #print("temp_list to append:", temp_list)
                            #print("temp_df_row.index: ",temp_df_row.index[0])
                            temp_df_row.set_value(temp_df_row.index[0], column_number, temp_list)
                            df = df.append(temp_df_row)
                            #print("temp_df_row",temp_df_row,"end")
                            #print(df.append(temp_df_row))
                            column_number+=1
                            if df.index[1] >= column_number:
                                temp_list = temp_df_row[column_number]
                                temp_list = temp_list.tolist()
                                temp_list = temp_list[0]
                            else:
                                flag = 1
                            if temp_list is None:
                                flag = 1
                                #print("game over for row number, colnum:", cut_side+'_'+str(i)+'_'+name, column_number)
                        cv2.imwrite(str(path)+'_sliced/'+cut_side+'_'+str(i)+'_'+name,corped_img)
                        ##script to show the cropped image
                        #plt.imshow(corped_img, cmap = 'gray', interpolation = 'bicubic')
                        #plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
                        #plt.show()
    df.to_pickle(new_df_path)

os.chdir('/home/ec2-user/table_detection/json2df/VOC2007/JPEGImages')
cwd = '/home/ec2-user/table_detection/json2df/VOC2007/JPEGImages'
slice_image_loop(cwd)